<div style="font-size:18pt; padding-top:20px; text-align:center">СЕМИНАР. <b>Комбинация решающих деревьев и </b> <span style="font-weight:bold; color:green">Sklearn + XGBoost</span></div><hr>
<div style="text-align:right;">Папулин С.Ю. <span style="font-style: italic;font-weight: bold;">(papulin.study@yandex.ru)</span></div>

<a name="0"></a>
<div><span style="font-size:14pt; font-weight:bold">Содержание</span>
    <ol>
        <li><a href="#1">Загрузка исходных данных</a></li>
        <li><a href="#2">Преобразование исходных данных</a></li>
        <li><a href="#3">Решающие деревья и выбор модели</a>
            <ol style = "list-style-type:lower-alpha">
                <li><a href="#3a">Формирование обучающего и тестового подмножеств</a></li>
                <li><a href="#3b">Вычисление базовой отметки</a></li>
                <li><a href="#3c">Random Forest и выбор модели</a></li>
                <li><a href="#3d">Gradient-boosted tree и выбор модели</a></li>
                <li><a href="#3e">XGBoost и Sklearn: Extreme Gradient boosting и выбор модели</a></li>
                <li><a href="#3f">XGBoost: Extreme Gradient boosting</a></li>
            </ol>
        </li>
        <li><a href="#4">Источники</a></li>
    </ol>
</div>

<p>Подключение стилей оформления</p>

In [1]:
%%html
<link href="css/style.css" rel="stylesheet" type="text/css">

<p><b>Подключение библиотек</b></p>

In [ ]:
import numpy as np
import pandas as pnd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

<a name="1"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">1. Загрузка исходных данных</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

Набор данных: [Black Friday](https://www.kaggle.com/sdolezel/black-friday)

In [ ]:
file_path = "../data/purchase/train.csv"

In [ ]:
df_data = pnd.read_csv(file_path, sep=",")
df_data.head(5)

In [ ]:
df_data.dtypes

<a name="2"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">2. Преобразование исходных данных</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

<p>Выбор исходных столбцов</p>

In [ ]:
df_use_data = df_data[["Marital_Status", "Product_Category_1", "Purchase"]]
df_use_data.head(5)

<p>Заполнение ячеек с неопределенными значениями</p>

In [ ]:
df_data.head(5)

In [ ]:
df_data.fillna(value=0, axis="columns")

In [ ]:
df_use_data["Product_Category_2"] = df_data["Product_Category_2"].fillna(0)
df_use_data["Product_Category_3"] = df_data["Product_Category_3"].fillna(0)
df_use_data.head(5)

<p>Преобразование для столбцов Gender, Stay_In_Current_City_Years и Age</p>

In [ ]:
age_ranges = df_data["Age"].unique()
dict_age_range = dict(zip(sorted(age_ranges), np.arange(0, len(age_ranges), 1.0)))
dict_age_range

In [ ]:
def convert_gender(x):
    if x == "F":
        return 1.0
    return 0.0

def convert_stay(x):
    if x == "4+":
        return 5.0
    try:
        y = float(x)
    except ValueError:
        return None
    else:
        return y

def convert_age(age_range):
    def convert(x):
        if x in age_range:
            return age_range[x]
        return None
    return convert

# df_use_data["GenderIndex"] = df_data.loc[:, "Gender"].apply(convert_gender)
df_use_data["GenderIndex"] = df_data["Gender"].apply(convert_gender)
df_use_data["StayIndex"] = df_data["Stay_In_Current_City_Years"].apply(convert_stay)
df_use_data["AgeIndex"] = df_data["Age"].apply(convert_age(dict_age_range))

df_use_data.head(5)

<p>Преобразование категориального признака City_Category в матрицу дискретных значений</p>

In [ ]:
df_use_data = df_use_data.join(pnd.get_dummies(df_data["City_Category"]))
df_use_data.head(5)

<p>Преобразование категориального признака Occupation в матрицу дискретных значений</p>

In [ ]:
df_use_data = df_use_data.join(pnd.get_dummies(df_data["Occupation"], prefix="Occ"))
df_use_data.head(5)

<p>Формирование вектора признаков</p>

In [ ]:
selected_columns = ["Marital_Status", "Product_Category_1", "Product_Category_2", "Product_Category_3",
                   "GenderIndex", "AgeIndex", "StayIndex", "A", "B", "C"] + ["Occ_"+str(el) for el in range(21)]

features = df_use_data[selected_columns].values
purchase = df_use_data["Purchase"].values

In [ ]:
features

In [ ]:
purchase

<a name="3"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">3. Решающие деревья и выбор модели</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

<a name="3a"></a>
<div style="display:table; width:100%">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-style:italic; font-weight:bold; font-size:12pt">
            a. Формирование обучающего и тестового подмножеств
        </div>
        <div style="display:table-cell; border:1px solid lightgrey; width:20%">
            <div style="display:table-cell; width:10%; text-align:center; background-color:whitesmoke;">
                <a href="#3">Назад</a>
            </div>
            <div style="display:table-cell; width:10%; text-align:center;">
                <a href="#3b">Далее</a>
            </div>
        </div>
    </div>
</div>

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(features, purchase, test_size=0.2, random_state=12)

In [ ]:
x_train.shape, x_test.shape

<a name="3b"></a>
<div style="display:table; width:100%">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-style:italic; font-weight:bold; font-size:12pt">
            b. Вычисление базовой отметки
        </div>
        <div style="display:table-cell; border:1px solid lightgrey; width:20%">
            <div style="display:table-cell; width:10%; text-align:center; background-color:whitesmoke;">
                <a href="#3a">Назад</a>
            </div>
            <div style="display:table-cell; width:10%; text-align:center;">
                <a href="#3c">Далее</a>
            </div>
        </div>
    </div>
</div>

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
purchase_mean = np.mean(y_train)
purchase_mean

In [ ]:
y_pred = np.empty(len(y_test))
y_pred.fill(purchase_mean)
y_pred[:5]

In [ ]:
mean_squared_error(y_test, y_pred)**(0.5)

In [ ]:
r2_score(y_test, y_pred)

<a name="3c"></a>
<div style="display:table; width:100%">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-style:italic; font-weight:bold; font-size:12pt">
            c. Sklearn: Random Forest и выбор модели
        </div>
        <div style="display:table-cell; border:1px solid lightgrey; width:20%">
            <div style="display:table-cell; width:10%; text-align:center; background-color:whitesmoke;">
                <a href="#3b">Назад</a>
            </div>
            <div style="display:table-cell; width:10%; text-align:center;">
                <a href="#3d">Далее</a>
            </div>
        </div>
    </div>
</div>

In [ ]:
from sklearn.ensemble import RandomForestRegressor

<p>Создание модели</p>

In [ ]:
rf_model = RandomForestRegressor(n_estimators=20, criterion="mse", max_depth=15, random_state=0)
rf_model

<p>Обучение</p>

In [ ]:
rf_model.fit(x_train, y_train)

<p>Отображение значений важности признаков</p>

In [ ]:
rf_model.feature_importances_

<p>Предсказание для тестового подмножества</p>

In [ ]:
y_pred_rf = rf_model.predict(x_test)
y_pred_rf

<p>Тестирование</p>

<p><i>RMSE</i></p>

In [ ]:
mean_squared_error(y_test, y_pred_rf)**(0.5)

<p><i>R^2</i></p>

In [ ]:
r2_score(y_test, y_pred_rf)

<p><b>Выбор модели</b></p>

<p>Создание базовой модели</p>

In [ ]:
rf_model = RandomForestRegressor(criterion="mse", random_state=0, verbose=True)

<p>Формирование сетки параметров для моделей</p>

In [ ]:
params = {"n_estimators": [10, 20], "max_depth": [10, 20]}

<p>Конфигурирования исходных данных для выбора модели с использованием кросс-валидации с k-folds</p>

In [ ]:
rf_gs = GridSearchCV(estimator=rf_model, param_grid=params, cv=4)

<p>Запуск процесса выбора модели по заданной сетке параметров</p>

In [ ]:
rf_gs.fit(x_train, y_train)

<div class="msg-block msg-info">
  <p class="msg-text-info">Процесс может занять около 10 мин.</p>
</div>

<p>Отображение значений ошибок для всех моделей</p>

In [ ]:
rf_gs.cv_results_

<p>Доступ к лучшей модели</p>

In [ ]:
rf_gs_best_model = rf_gs.best_estimator_
rf_gs_best_model

<p>Тестирование лучшей модели (без повторного обучения на x_train) - R^2</p>

In [ ]:
rf_gs_best_model.score(x_test, y_test)

<a name="3d"></a>
<div style="display:table; width:100%">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-style:italic; font-weight:bold; font-size:12pt">
            d. Sklearn: Gradient boosting и выбор модели
        </div>
        <div style="display:table-cell; border:1px solid lightgrey; width:20%">
            <div style="display:table-cell; width:10%; text-align:center; background-color:whitesmoke;">
                <a href="#3c">Назад</a>
            </div>
            <div style="display:table-cell; width:10%; text-align:center;">
                <a href="#3e">Далее</a>
            </div>
        </div>
    </div>
</div>

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

<p>Создание модели</p>

In [ ]:
gb_model = GradientBoostingRegressor(n_estimators=40, loss="ls", 
                                     criterion="mse", max_depth=5, 
                                     alpha=0.1, random_state=0, verbose=True)

<p>Обучение</p>

In [ ]:
gb_model.fit(x_train, y_train)

<p>Отображение значений важности признаков</p>

In [ ]:
gb_model.feature_importances_

<p>Предсказание для тестового подмножества</p>

In [ ]:
y_pred_gb = gb_model.predict(x_test)
y_pred_gb

<p>Тестирование</p>

<p><i>RMSE</i></p>

In [ ]:
mean_squared_error(y_test, y_pred_gb)**(0.5)

<p><i>R^2</i></p>

In [ ]:
r2_score(y_test, y_pred_gb)

<p><b>Выбор модели</b></p>

<p>Создание базовой модели</p>

In [ ]:
gb_model = GradientBoostingRegressor(loss="ls", criterion="mse", alpha=0.1, random_state=0, verbose=True)

<p>Формирование сетки параметров для моделей</p>

In [ ]:
params = {"n_estimators": [10, 20], "max_depth": [2, 5]}

<p>Конфигурирования исходных данных для выбора модели с использованием кросс-валидации с k-folds</p>

In [ ]:
gb_gs = GridSearchCV(estimator=gb_model, param_grid=params, cv=4)

<p>Запуск процесса выбора модели по заданной сетке параметров</p>

In [ ]:
gb_gs.fit(x_train, y_train)

<div class="msg-block msg-info">
  <p class="msg-text-info">Процесс может занять 10-15 мин.</p>
</div>

<p>Отображение значений ошибок для всех моделей</p>

In [ ]:
gb_gs.cv_results_

<p>Доступ к лучшей модели</p>

In [ ]:
gb_gs_best_model = gb_gs.best_estimator_
gb_gs_best_model

<p>Тестирование лучшей модели (без повторного обучения на x_train)</p>

In [ ]:
gb_gs_best_model.score(x_test, y_test)

<a name="3e"></a>
<div style="display:table; width:100%">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-style:italic; font-weight:bold; font-size:12pt">
            e. XGBoost и Sklearn: Extreme Gradient boosting и выбор модели
        </div>
        <div style="display:table-cell; border:1px solid lightgrey; width:20%">
            <div style="display:table-cell; width:10%; text-align:center; background-color:whitesmoke;">
                <a href="#3d">Назад</a>
            </div>
            <div style="display:table-cell; width:10%; text-align:center;">
                <a href="#3f">Далее</a>
            </div>
        </div>
    </div>
</div>

<div class="msg-block msg-warning">
  <p class="msg-text-warn">Установите xgboost:</p>
  <p class="code-block code-font"><span class="code-key">pip</span> install xgboost</p>
</div>

In [ ]:
import xgboost as xgb

<p>Создание модели</p>

In [ ]:
xgb_reg_model = xgb.XGBRegressor(n_estimators=40, max_depth=5, max_delta_step=0, reg_alpha=0)

<p>Обучение</p>

In [ ]:
xgb_reg_model.fit(x_train, y_train)

<p>Предсказание для тестового подмножества</p>

In [ ]:
y_pred = xgb_reg_model.predict(x_test)

<p><i>RMSE</i></p>

In [ ]:
mean_squared_error(y_test, y_pred)**(0.5)

<p><i>R^2</i></p>

In [ ]:
r2_score(y_test, y_pred)

<p>или</p>

In [ ]:
xgb_reg_model.score(x_test, y_test)

<p><b>Выбор модели</b></p>

<p>Создание базовой модели</p>

In [ ]:
xgb_reg_model = xgb.XGBRegressor(n_estimators=40, max_delta_step=0, reg_alpha=0)

<p>Формирование сетки параметров для моделей</p>

In [ ]:
params = {"max_depth": [5, 10], "min_child_weight": [1, 10]}

<p>Конфигурирования исходных данных для выбора модели с использованием кросс-валидации с k-folds</p>

In [ ]:
xgd_gs = GridSearchCV(estimator=xgb_reg_model, param_grid=params, cv=4)

<p>Запуск процесса выбора модели по заданной сетке параметров</p>

In [ ]:
xgd_gs.fit(x_train, y_train)

<div class="msg-block msg-info">
  <p class="msg-text-info">Процесс может занять около 10 мин.</p>
</div>

<p>Отображение значений ошибок для всех моделей</p>

In [ ]:
xgd_gs.cv_results_

<p>Доступ к лучшей модели</p>

In [ ]:
xgd_gs_best_model = xgd_gs.best_estimator_
xgd_gs_best_model

<p>Тестирование лучшей модели (без повторного обучения на x_train) - R^2</p>

In [ ]:
xgd_gs_best_model.score(x_test, y_test)

<a name="3f"></a>
<div style="display:table; width:100%">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-style:italic; font-weight:bold; font-size:12pt">
            f. XGBoost: Extreme Gradient boosting
        </div>
        <div style="display:table-cell; border:1px solid lightgrey; width:20%">
            <div style="display:table-cell; width:10%; text-align:center; background-color:whitesmoke;">
                <a href="#3e">Назад</a>
            </div>
            <div style="display:table-cell; width:10%; text-align:center;">
                <a href="#4">Далее</a>
            </div>
        </div>
    </div>
</div>

<p>Преобразование матрицы признаков в разреженный вид</p>

In [ ]:
from scipy import sparse

In [ ]:
x_train_sparse = sparse.csr_matrix(x_train)
x_test_sparse = sparse.csr_matrix(x_test)
x_train_sparse

In [ ]:
x_train_dmatrix = xgb.DMatrix(x_train_sparse, label=y_train, missing = 0)
x_test_dmatrix = xgb.DMatrix(x_test_sparse, label=y_test, missing = 0)

<p>Создание модели</p>

In [ ]:
# Параметры модели
param = {"max_depth": 5, "eta": 1, "silent": 1, "objective": "reg:linear"}
num_round = 40

<p>Обучение</p>

In [ ]:
xgb_model = xgb.train(param, x_train_dmatrix, num_round)
xgb_model

<p>Отображение значений важности признаков</p>

In [ ]:
xgb_model.get_score(importance_type="weight")

<p>Предсказание для тестового подмножества</p>

In [ ]:
y_pred_xgb = xgb_model.predict(x_test_dmatrix)
y_pred_xgb

<p>RMSE</p>

In [ ]:
mean_squared_error(y_test, y_pred_xgb)**(0.5)

<p>R^2</p>

In [ ]:
r2_score(y_test, y_pred_xgb)

<p><b>Отображения результата</b></p>

In [ ]:
plot = xgb.plot_importance(xgb_model)

In [ ]:
import graphviz

<div class="msg-block msg-warning">
  <p class="msg-text-warn">Устанавливайте graphviz через следующую команду:</p>
  <p class="code-block code-font">sudo <span class="code-key">apt-get</span> install graphviz</p>
</div>

In [ ]:
xgb.to_graphviz(xgb_model, num_trees=1)

<p><b>Кросс-валидация с k-folds</b></p>

In [ ]:
# Параметры модели
param = {"max_depth": 5, "eta": 1, "silent": 1, "objective": "reg:linear" }
num_round = 40

In [ ]:
xgb.cv(param, x_train_dmatrix, num_round, nfold=4,
       metrics={"rmse"}, seed = 0,
       callbacks=[xgb.callback.print_evaluation(show_stdv=True)])

<a name="4"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">4. Источники</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

In [ ]:
https://datahack.analyticsvidhya.com/contest/black-friday/
https://www.analyticsvidhya.com/blog/2016/05/h2o-data-table-build-models-large-data-sets/
https://github.com/dmlc/xgboost/tree/master/demo/guide-python
http://xgboost.readthedocs.io/en/latest/python/python_intro.html
https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/